In [135]:
import pandas as pd
from tqdm import tqdm

In [62]:
df_payments = pd.DataFrame()
df_providers = pd.DataFrame()
df_ex_rates = pd.read_csv('data/ex_rates.csv')

for i in range(1, 2):
    payments_data = pd.read_csv(f'data/payments_{i}.csv')
    providers_data = pd.read_csv(f'data/providers_{i}.csv')

    df_payments = pd.concat([df_payments, payments_data], ignore_index=True)
    df_providers = pd.concat([df_providers, providers_data], ignore_index=True)

df_providers.drop(['LIMIT_BY_CARD'], axis=1, inplace=True)


In [63]:
df_payments

,eventTimeRes,amount,cur,payment,cardToken
0,2024-11-25 20:59:59,90000.00,UZS,7d1476c9f1eedcd4a2083cc399ec1c1b,7ba0e52864c29fe2fcc1977be1ffc8d5
1,2024-11-25 20:59:58,2700.00,RUB,6eb49e22afdd28083ff82cdff8068880,774d551c9fff62fbac6c4ef424af0446
2,2024-11-25 20:59:58,2000.00,RUB,0e2705d1083b67f2c5ea12de3b7ab182,a3481a01b54d0c96dd44200389888bc6
3,2024-11-25 20:59:58,27.27,AZN,c30b3850e85454145bcb0dda3d209d88,61e3567ab56a79911fb7bcc0cc13db64
4,2024-11-25 20:59:58,8.00,AZN,4af690ba4eebb6d4782d9a3051dc58d2,373e0a66a63b8661f10afd098c68aeab
...,...,...,...,...,...
193973,2024-11-24 21:00:01,90.00,BRL,f3adde51de36a39508d11d4bf4a98edc,336d5ebc5436534e61d16e63ddfca327
193974,2024-11-24 21:00:01,36.00,BRL,d8a90bf2cc88aa672b30bf1b4959751a,336d5ebc5436534e61d16e63ddfca327
193975,2024-11-24 21:00:01,35.00,BRL,90a88e0d5b6bfff12909878f265cc2f4,336d5ebc5436534e61d16e63ddfca327
193976,2024-11-24 21:00:01,180.00,BRL,0ab79b6d41d6e887e84ec4a85ea33733,336d5ebc5436534e61d16e63ddfca327


In [64]:
df_providers

,TIME,ID,CONVERSION,AVG_TIME,MIN_SUM,MAX_SUM,LIMIT_MIN,LIMIT_MAX,COMMISSION,CURRENCY
0,2024-11-24 21:00:00,0,0.50,18.0,700.0,5500.0,61000.0,12900000.0,0.035,UZS
1,2024-11-24 21:00:00,1,0.65,8.0,900.0,9000.0,21000.0,18000000.0,0.035,UZS
2,2024-11-24 21:00:00,2,0.50,18.0,400.0,8500.0,51000.0,13400000.0,0.030,RUB
3,2024-11-24 21:00:00,3,0.50,16.0,900.0,6000.0,61000.0,19800000.0,0.045,RUB
4,2024-11-24 21:00:00,4,0.70,26.0,300.0,6500.0,81000.0,23600000.0,0.050,RUB
...,...,...,...,...,...,...,...,...,...,...
506,2024-11-25 21:00:00,29,0.50,8.0,200.0,8500.0,41000.0,18200000.0,0.035,USD
507,2024-11-25 21:00:00,32,0.50,20.0,800.0,9500.0,1000.0,9800000.0,0.035,USD
508,2024-11-25 21:00:00,36,0.50,14.0,300.0,6000.0,21000.0,9900000.0,0.050,GHS
509,2024-11-25 21:00:00,38,0.50,24.0,800.0,4500.0,21000.0,32700000.0,0.035,GHS


In [65]:
df_payments['amount'].describe()

count    1.939780e+05
mean     5.946773e+04
std      7.840276e+05
min      7.000000e-01
25%      9.000000e+01
50%      1.845000e+03
75%      2.000000e+04
max      2.070001e+08
Name: amount, dtype: float64

In [66]:
df_payments.iloc[0]

eventTimeRes                 2024-11-25 20:59:59
amount                                   90000.0
cur                                          UZS
payment         7d1476c9f1eedcd4a2083cc399ec1c1b
cardToken       7ba0e52864c29fe2fcc1977be1ffc8d5
Name: 0, dtype: object

In [67]:
a = dict(df_ex_rates.values)
a = dict((v, k) for k, v in a.items())

df_payments['amount_in_USD'] = df_payments['cur'].map(lambda x: a[x]) * df_payments['amount']

df_providers = df_providers.rename(columns={'CURRENCY': 'cur'})
df_providers['MIN_SUM_in_USD'] = df_providers['cur'].map(lambda x: a[x]) * df_providers['MIN_SUM']
df_providers['MAX_SUM_in_USD'] = df_providers['cur'].map(lambda x: a[x]) * df_providers['MAX_SUM']
df_providers['LIMIT_MIN_in_USD'] = df_providers['cur'].map(lambda x: a[x]) * df_providers['LIMIT_MIN']
df_providers['LIMIT_MAX_in_USD'] = df_providers['cur'].map(lambda x: a[x]) * df_providers['LIMIT_MAX']

In [68]:
df_payments['amount_in_USD'].describe()

count    193978.000000
mean         21.409089
std          79.056895
min           0.008628
25%           4.137192
50%           7.800000
75%          17.299741
max       16146.003931
Name: amount_in_USD, dtype: float64

In [71]:
df_providers['MAX_SUM_in_USD'].describe()

count     511.000000
mean     1904.394975
std      2442.746097
min         0.273000
25%       105.789500
50%       671.436000
75%      3000.000000
max      9897.375500
Name: MAX_SUM_in_USD, dtype: float64

In [72]:
for i in df_payments:
    print(i)

eventTimeRes
amount
cur
payment
cardToken
amount_in_USD


In [105]:
def set_lim_min_max(df_providers):
    df_providers['TIME'] = pd.to_datetime(df_providers['TIME'])

    df_providers['DATE'] = df_providers['TIME'].dt.date

    first_entries = df_providers.groupby(['ID', 'DATE']).first().reset_index()
    first_entries = first_entries[['ID', 'DATE', 'LIMIT_MIN_in_USD', 'LIMIT_MAX_in_USD']]

    first_entries = first_entries.rename(
        columns={'LIMIT_MIN_in_USD': 'FIRST_LIMIT_MIN', 'LIMIT_MAX_in_USD': 'FIRST_LIMIT_MAX'})

    df_providers = df_providers.merge(first_entries, on=['ID', 'DATE'], how='left')

    df_providers['LIMIT_MIN_in_USD'] = df_providers['FIRST_LIMIT_MIN']
    df_providers['LIMIT_MAX_in_USD'] = df_providers['FIRST_LIMIT_MAX']

    df_providers = df_providers.drop(columns=['FIRST_LIMIT_MIN', 'FIRST_LIMIT_MAX', 'DATE'])
    return df_providers


df_providers = set_lim_min_max(df_providers)

In [161]:
df_payments['TIME'] = pd.to_datetime(df_payments['eventTimeRes'])

20

In [188]:
df_payments['TIME'] = pd.to_datetime(df_payments['TIME'])
df_providers['TIME'] = pd.to_datetime(df_providers['TIME'])

providers_by_cur = {cur: group.copy() for cur, group in df_providers.groupby('cur')}

for i, row in tqdm(df_payments.iterrows()):
    temp_providers = providers_by_cur[row['cur']]

    temp_providers = temp_providers.copy()
    temp_providers.loc[:, 'TIME_DIFF'] = (temp_providers['TIME'] - row['TIME'])

    temp_providers = temp_providers[(temp_providers['TIME_DIFF'].dt.total_seconds() <= 0)]

    min_time_diff = temp_providers['TIME_DIFF'].max()

    closest_provider = temp_providers[
        (temp_providers['TIME_DIFF'] == min_time_diff)]

61288it [00:29, 2060.30it/s]


KeyboardInterrupt: 